In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from dateutil.relativedelta import relativedelta

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
database_path = "Resources/hawaii.sqlite"
engine = create_engine(f"sqlite:///{database_path}")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

In [ ]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# find latest date
latest_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first().date
# format the string
latest_date = dt.datetime.strptime(latest_date, '%Y-%m-%d')
# get the date 1 year before
last_year = latest_date - relativedelta(months=+12)
# show date twelve months before
last_year

In [ ]:
# query for the last 12 months of precipitation data
last_twelve_months = session.query(Measurement.date, func.avg(Measurement.prcp)).filter(Measurement.date >= last_year).group_by(Measurement.date).all()

In [ ]:
# saving results to dataframe
prcp_df = pd.DataFrame(last_twelve_months, columns=['date', 'precipitation'])
# set date to index
prcp_df.set_index('date', inplace=True)
# show dataframe
prcp_df.head()

In [ ]:
# create lists of date and precipitation
date = prcp_df.index
prcp = prcp_df['precipitation']

In [ ]:
# create the figure
fig, ax = plt.subplots()
# plot the data
ax.bar(date, prcp, label="precipitation", width=3)
# clear xticklabels
ax.set_xticklabels("")
# set title
ax.set_title("Precipitation data: last 12 months")
# set xlabel
ax.set_xlabel("date")
# set ylabel
ax.set_ylabel("precipitation (in inches)")
# create a legend
ax.legend()
# show the plot
plt.show()

In [ ]:
# display summary statistics
prcp_df.describe()

In [ ]:
# query to obtain the number of stations
number_of_stations = session.query(Measurement.station).distinct(Measurement.station).count()
# show number of stations
number_of_stations

In [ ]:
# query to find the most active stations
most_active_stations = session.query(Measurement.station, func.count(Measurement.station)).group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()
# show the most active stations
most_active_stations

In [ ]:
# find most active station
most_active_station = most_active_stations[0][0]
# find min, max, and avg temps for this station
most_active_station_max_min_avg = session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).filter(Measurement.station == most_active_station).all()
# show the results
most_active_station_max_min_avg

In [ ]:
# query for the last 12 months of tobs for the most active station
most_active_station_data = session.query(Measurement.station, Measurement.tobs).filter(Measurement.station == most_active_station).filter(Measurement.date >= last_year).all()
# create dataframe
most_active_station_data_df = pd.DataFrame(most_active_station_data, columns=['station', 'Temperature Observations'])
# set station as index
most_active_station_data_df.set_index('station', inplace=True)
# show dataframe
most_active_station_data_df.head()

In [ ]:
# plot the results as a histogram
# create the figure
fig, ax = plt.subplots()
# plot the data
ax.hist(most_active_station_data_df['Temperature Observations'], label='temperature observations')
# create a legend
ax.legend()
# show the plot
plt.show()

In [ ]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    return session.query(func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()

# function usage example
print(calc_temps('2012-02-28', '2012-03-05'))

In [ ]:
trip_temps = calc_temps('2016-02-28', '2016-03-05')
trip_temps

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(trip_temps)
ax.set_title('Trip Avg Temp')
plt.show()

In [ ]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
start_date = '2012-02-28'
end_date = '2012-03-05'
rainfall_per_station = session.query(Measurement.station, Station.name, Station.latitude, Station.longitude, Station.elevation, func.sum(Measurement.prcp)).group_by(Measurement.station).filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).order_by(func.sum(Measurement.prcp).desc()).all()
rainfall_per_station